In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import csv
from dateutil import parser
from datetime import datetime
pd.set_option("display.max_colwidth", None)

In [18]:
def date_extractor (date_time_str):
  date_pattern = re.compile(r'(\d{2})/(\d{2})/(\d{4})')
  match = date_pattern.search(date_time_str)

  if match:
      day = match.group(1)
      month = match.group(2)
      year = match.group(3)

      # Parse the time part
      time_part = date_time_str.split('|')[1].strip()
      hour, minute = map(int, time_part.split(':'))

      # Create a datetime object
      parsed_datetime = datetime(int(year), int(month), int(day), hour, minute)
      return(parsed_datetime.date())
print(date_extractor("17/09/2023 | 07:22"))


2023-09-17


In [19]:
def crawl_by_keyword(keyword):
    df=pd.read_csv('tienphong_'+keyword+'.csv')
    keyword_lookup = keyword
    csv_file = open('tienphong_'+keyword+'_extracted.csv', 'w', newline='',encoding='utf-8')
    writer = csv.writer(csv_file)
    writer.writerow(['Title', 'URL','Publish Date', 'Mentioned Times','Word Count'])

    for index, row in df.iloc[0:].iterrows():
        title = row['Title']
        url = row['URL']
        news_url = url
        response = requests.get(news_url)
        if response.status_code == 403:
            break
        write_flag = True

        soup=BeautifulSoup(response.content,'html.parser')
        try: 
            publish_date = soup.find('time', attrs={'datetime': True}).get_text()
            cleaned_publish_date = date_extractor(publish_date)
        except AttributeError:
            print('Could not get date of publication of: \n' + str(news_url))
            write_flag =False

        try:
            news_content = soup.find('div',class_='col-27 article-content').get_text()
            nofm = news_content.lower().count(keyword_lookup.lower())
            nofw = len(news_content.lower().split())
        except  AttributeError:
            print('Cant target news content of: \n' + str(news_url))
            try:
                
                news_content = soup.find('article',class_='col article').get_text()
                nofm = news_content.lower().count(keyword_lookup.lower())
                nofw = len(news_content.lower().split())
                print('Target article instead successfully \n')
            except:
                print('Cant target news content article of: \n' + str(news_url) + '\n')
                write_flag = False

        try:
            # Extract the date component
            date_only = cleaned_publish_date
        except ValueError:
            print("Invalid time format. of: \n" + str(news_url))
        if write_flag == True:
            writer.writerow([title, url,date_only,nofm,nofw])
    csv_file.close()

In [20]:
keywords = ["số hóa"]
for text in keywords:
    crawl_by_keyword(text)

Cant target news content of: 
https://tienphong.vn/cong-nghe-so-ai-va-van-de-dao-duc-vao-ban-nghi-su-post1569601.tpo
Target article instead successfully 

Cant target news content of: 
https://tienphong.vn/bo-truong-tttt-nguyen-manh-hung-tri-tue-nhan-tao-da-san-sang-tao-ra-tri-thuc-post1571361.tpo
Target article instead successfully 

Cant target news content of: 
https://tienphong.vn/sinh-vien-gen-z-hao-huc-hien-ke-cho-hoi-sinh-vien-post1533211.tpo
Target article instead successfully 

Cant target news content of: 
https://tienphong.vn/hoi-nghi-nghi-si-tre-toan-cau-lan-thu-9-luat-hoa-hanh-vi-bao-luc-tren-khong-gian-mang-de-xu-ly-post1569421.tpo
Target article instead successfully 

Cant target news content of: 
https://tienphong.vn/chuyen-doi-so-cho-chung-ta-oxy-de-vuot-qua-nhung-thach-thuc-post1569289.tpo
Target article instead successfully 



KeyboardInterrupt: 